In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

In [2]:
word_to_idx = {"hello":0, "world": 1}
embeds = nn.Embedding(2,5)
lookup_tensor = torch.tensor([word_to_idx["hello"]], dtype=torch.long)
hello_embeds = embeds(lookup_tensor)

In [3]:
hello_embeds

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)

In [4]:
CONTEXT_SIZE = 2    #because of trigram dataset
EMBEDDING_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [5]:
trigrams=[([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence)-2)]

In [6]:
from pprint import pprint

In [7]:
vocab = set(test_sentence)
word_to_idx = {word:idx for idx,word in enumerate(vocab)}


In [8]:
pprint(sorted(word_to_idx.items(), key = lambda x: x[1]))

[('be', 0),
 ('shame,', 1),
 ('cold.', 2),
 ('eyes,', 3),
 ('art', 4),
 ("excuse,'", 5),
 ('all', 6),
 ('Then', 7),
 ('child', 8),
 ('a', 9),
 ('fair', 10),
 ('were', 11),
 ('warm', 12),
 ('it', 13),
 ('all-eating', 14),
 ('How', 15),
 ('small', 16),
 ('praise.', 17),
 ('gazed', 18),
 ('days;', 19),
 ('thriftless', 20),
 ('within', 21),
 ('brow,', 22),
 ('so', 23),
 ('treasure', 24),
 ('make', 25),
 ('field,', 26),
 ('Shall', 27),
 ('to', 28),
 ('proud', 29),
 ('held:', 30),
 ('shall', 31),
 ("totter'd", 32),
 ('old', 33),
 ('lies,', 34),
 ('mine', 35),
 ('lusty', 36),
 ("feel'st", 37),
 ('beauty', 38),
 ('weed', 39),
 ('answer', 40),
 ('new', 41),
 ('succession', 42),
 ('my', 43),
 ('made', 44),
 ('thine', 45),
 ('couldst', 46),
 ('being', 47),
 ('and', 48),
 ("deserv'd", 49),
 ('praise', 50),
 ('And', 51),
 ('sum', 52),
 ('more', 53),
 ('the', 54),
 ('dig', 55),
 ('If', 56),
 ('his', 57),
 ('thou', 58),
 ('on', 59),
 ('forty', 60),
 ('besiege', 61),
 ('by', 62),
 ('To', 63),
 ('when'

In [9]:
class NGramlanguageModular(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramlanguageModular,self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # making lookup-table with word embedding
        self.linear1 = nn.Linear(context_size*embedding_dim, 128) 
        self.linear2 = nn.Linear(128, vocab_size)


    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1,-1)) #passing row vector of input data points
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out,1)
        return(log_probs)



In [10]:
losses = []
loss_function = nn.NLLLoss() #negative log likelihood

model = NGramlanguageModular(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [11]:
#training routine

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        
        model.zero_grad() #clearing leftover grads
        context_idxs = torch.tensor([word_to_idx[w] for w in context], dtype = torch.long) #making input row of vector with 2d context
        
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_idx[target]]))
        loss.backward()
        optimizer.step()

        total_loss +=loss.item()
    losses.append(total_loss)
pprint(f'Losses: {losses}', indent=4, width=4)

('Losses: '
 '[521.4588832855225, '
 '518.9084067344666, '
 '516.3773002624512, '
 '513.8643174171448, '
 '511.3696217536926, '
 '508.8914120197296, '
 '506.4289002418518, '
 '503.9818139076233, '
 '501.5498299598694, '
 '499.1313314437866]')


In [12]:
#practice exercise
EMBEDDING_DIM = 10
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

print(data[0])



(['We', 'are', 'to', 'study'], 'about')


In [13]:
#model class
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_dim):
        super(CBOW,self).__init__()
        self.embed_layer = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 =  nn.Linear(2*context_dim*embedding_dim, 128) #include 2 for double sided context here
        self.linear2 =  nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embed_layer(inputs).view((1,-1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out,1)
        return log_probs




In [14]:
#helper function

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


In [15]:
loss_function = nn.NLLLoss()
model = CBOW(vocab_size, EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(),lr=0.001)

In [16]:
final_losses =[]
for epoch in range(10):
    total_loss = 0
    for context, target in data:

        model.zero_grad()
     
        log_probs = model(make_context_vector(context,word_to_ix))
        target_vector = torch.tensor([word_to_ix[target]],dtype=torch.long)
        loss = loss_function(log_probs,target_vector)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    final_losses.append(total_loss)

pprint(f'final_losses: {final_losses}', indent=2, width=2)

('final_losses: '
 '[226.61858677864075, '
 '225.16838121414185, '
 '223.72908401489258, '
 '222.29899215698242, '
 '220.8776319026947, '
 '219.46639919281006, '
 '218.06447553634644, '
 '216.66943383216858, '
 '215.28110694885254, '
 '213.8991014957428]')
